In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from proj1_helpers import *
from costs import *
from lab_helpers import *
from plots_lab4 import *
from my_helpers import *
import pandas as pd
import matplotlib.pyplot as plt

## To Do Graphs
- PCA Decomposition + Classification
- Correlation
- Changing loss for different algorithms
- ???

## To Do tasks
- Manuel: implement K-fold to choose degree, change split data , deal with outliers, adding log colm (?)
- Gabbo: ?
- Marco: ?

## Load the training data into feature matrix, class labels, and event ids:

In [26]:
DATA_TRAIN_PATH = '../data/train.csv'
y_or, tX_or, ids = load_csv_data(DATA_TRAIN_PATH)

In [27]:
y = y_or
tX = tX_or

In [28]:
#for logistic
y = y_for_logistic(y)

## Data preprocessing

In [29]:
x_tr_tot, x_te_tot, y_tr_tot, y_te_tot = split_data(y,tX,0.8,1)

In [30]:
y_tr, tX_tr, indexes_tr, means_tr, std_tr = preprocessing(y_tr_tot, x_tr_tot)

In [31]:
y_te, tX_te, indexes_te, means_te, std_te = preprocessing(y_te_tot, x_te_tot)

In [45]:
def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood."""
    ln = np.log(1 + np.exp(tx.dot(w)))
    s = y.T.dot(tx).dot(w)
    loss = np.sum(ln) - s
    if (np.isinf(loss)):
        loss = np.finfo(loss.dtype).max
    return loss

In [33]:
def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    return tx.T.dot(sigmoid(tx.dot(w)) - y)

In [34]:
def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descent using logistic regression.
    Return the loss and the updated w.
    """
    grad = calculate_gradient(y, tx, w)
    loss = calculate_loss(y, tx, w)
    w = w - gamma * grad
    return loss, w

In [56]:
def logistic_regression_gradient_descent(y, x, gamma):
    # init parameters
    max_iter = 2000
    threshold = 1e-8
    #gamma = 0.00001
    losses = []

    # build tx
    #tx = np.c_[np.ones((y.shape[0],)), x]
    tx = x
    w = 0.1 * np.ones((tx.shape[1],))

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tx, w, gamma)
        # log info
        #if iter % 10 == 0:
            #print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    #print("loss={l}".format(l=calculate_loss(y, tx, w)))
    return loss, w

In [60]:
list_loss = []
list_w = []
for jet in range(0,8):
    tX_tr_poly = build_poly(tX_tr[jet], degrees_star[jet])
    loss, w = logistic_regression_gradient_descent(y_tr[jet], tX_tr_poly, lambdas_star[jet])
    list_loss.append(loss)
    list_w.append(w)

In [61]:
#cell for logistic
degrees = [3, 3, 3, 3, 3, 3, 3, 3]
y_pred = build_predictions(tX_te, indexes_te, list_w, degrees_star, logistic=True) 
acc = accuracy(y_te_tot, y_pred)
print(acc)

0.5905


In [15]:
def logistic_regression_sub_gradient_descent(y, x):
    # init parameters
    max_iter = 10000
    threshold = 1e-8
    gamma = 0.05
    losses = []

    # build tx
    #tx = np.c_[np.ones((y.shape[0],)), x]
    tx = x
    w = np.ones((tx.shape[1],))

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        for yn, xn in batch_iter(y, tx, 1):
            loss, w = learning_by_gradient_descent(yn, xn, w, gamma)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    #print("loss={l}".format(l=calculate_loss(y, tx, w)))
    return loss, w

In [16]:
list_loss = []
list_w = []
for jet in range(0,8):
    tX_tr_poly = build_poly(tX_tr[jet], 7)
    loss, w = logistic_regression_sub_gradient_descent(y_tr[jet], tX_tr_poly)
    list_loss.append(loss)
    list_w.append(w)

Current iteration=0, loss=-1194.4782787261452
Current iteration=100, loss=-114356.27650051548
Current iteration=200, loss=-18686.662145914823
Current iteration=300, loss=-19046.66253190903
Current iteration=400, loss=-21374.594533107076
Current iteration=500, loss=-216682.43753599777
Current iteration=600, loss=-238221.3481878505


KeyboardInterrupt: 

In [71]:
#cell for logistic
tX_te_poly = list()
for t in tX_te:
    tX_te_poly.append(build_poly(t, 7))
y_pred = build_predictions(tX_te_poly, indexes_te, list_w, logistic=True) 
acc = accuracy(y_te_tot, y_pred)
print(acc)

0.62908


In [23]:
def calculate_hessian(y, tx, w):
    """return the hessian of the loss function."""
    N = tx.shape[0]
    S = np.zeros((N,N))
    for i in range(N):
        sigma = sigmoid(tx[i,:].T.dot(w))
        S[i,i] = sigma*(1-sigma)
    return tx.T.dot(S).dot(tx)

In [24]:
def logistic_regression(y, tx, w):
    """return the loss, gradient, and hessian."""
    return calculate_loss(y, tx, w), calculate_gradient(y, tx, w), calculate_hessian(y, tx, w)

In [25]:
def learning_by_newton_method(y, tx, w):
    """
    Do one step on Newton's method.
    return the loss and updated w.
    """
    # mettiamo il gamma???????
    gamma = 0.001
    loss, gradient, hessian = logistic_regression(y, tx, w)
    w = w - gamma * np.linalg.inv(hessian).dot(gradient)
    return loss, w

In [28]:
def logistic_regression_newton_method_stochastic(y, x):
    # init parameters
    max_iter = 100
    threshold = 1e-8
    lambda_ = 0.1
    losses = []

    # build tx
    #tx = np.c_[np.ones((y.shape[0],)), x]
    tx = x
    w = np.random.randn(tx.shape[1])
    
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        for yn, xn in batch_iter(y, tx, 10):
            loss, w = learning_by_newton_method(yn, xn, w)
            #print (xn.shape)
            #print(calculate_hessian(yn, xn, w).shape)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    return loss, w

In [29]:
list_loss = []
list_w = []
for jet in range(0,8):
    loss, w = logistic_regression_newton_method_stochastic(y_tr[jet], tX_tr[jet])
    list_loss.append(loss)
    list_w.append(w)

Current iteration=0, the loss=12.390456798923466


LinAlgError: Singular matrix

In [83]:
def penalized_logistic_regression(y, tx, w, lambda_):
    """return the loss, gradient, and hessian."""
    N = tx.shape[0]
    D = tx.shape[1]
    loss = calculate_loss(y, tx, w) + (lambda_ / 2) * np.linalg.norm(w)**2
    gradient = calculate_gradient(y, tx, w) + lambda_ * w
    
    return loss, gradient

In [57]:
def penalized_logistic_regression_newton_method(y, tx, w, lambda_):
    """return the loss, gradient, and hessian."""
    N = tx.shape[0]
    D = tx.shape[1]
    loss = calculate_loss(y, tx, w) + lambda_ / 2 * np.linalg.norm(w)
    gradient = calculate_gradient(y, tx, w) + lambda_ * w
    hessian = calculate_hessian(y, tx, w) + N * np.identity(D)
    return loss, gradient, hessian

In [64]:
def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
    """
    Do one step of gradient descent, using the penalized logistic regression.
    Return the loss and updated w.
    """
    #loss, gradient, hessian = penalized_logistic_regression(y, tx, w, lambda_)
    loss, gradient = penalized_logistic_regression(y, tx, w, lambda_)
    w = w - gamma * gradient
    return loss, w

In [101]:
def logistic_regression_penalized_gradient_descent(y, x):
    # init parameters
    max_iter = 10000
    gamma = 0.00001
    lambda_ = 0.1
    threshold = 1e-8
    losses = []

    # build tx
    #tx = np.c_[np.ones((y.shape[0],)), x]
    tx = x
    w = np.zeros((tx.shape[1],))

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)
        # log info
        if iter % 1000 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        #converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    return loss, w

In [102]:
list_loss = []
list_w = []
for jet in range(0,8):
    loss, w = logistic_regression_penalized_gradient_descent(y_tr[jet], tX_tr[jet])
    list_loss.append(loss)
    list_w.append(w)

Current iteration=0, loss=14466.674805466619
Current iteration=1000, loss=4072.4469655889584
Current iteration=2000, loss=3798.1914880938857
Current iteration=3000, loss=3699.928350177449
Current iteration=4000, loss=3658.2774047262965
Current iteration=5000, loss=3638.1839849118037
Current iteration=6000, loss=3627.21384693739
Current iteration=7000, loss=3620.3909907179204
Current iteration=8000, loss=3615.557576215821
Current iteration=9000, loss=3611.7247210138403
Current iteration=0, loss=40931.72730642589
Current iteration=1000, loss=30170.059065374655
Current iteration=2000, loss=29730.207217696887
Current iteration=3000, loss=29524.94845406814
Current iteration=4000, loss=29374.097390848463
Current iteration=5000, loss=29252.767903514865
Current iteration=6000, loss=29151.77002406338
Current iteration=7000, loss=29065.665918020914
Current iteration=8000, loss=28990.830131941442
Current iteration=9000, loss=28924.74557374494
Current iteration=0, loss=4160.962524901352
Current it

In [85]:
def cross_validation(y, x, k_indices, k, lambda_, gamma, acc = False, ls = False):
    """return the loss of ridge regression."""
    losses_tr = []
    losses_te = []
    accuracies = []
    ws = []
    
    for k_group in range(k):
        index_te = k_indices[k_group]
        index_tr = np.setdiff1d(np.arange(len(y)), index_te)
        #print(index_te, index_tr)
        x_te = x[index_te]
        x_tr = x[index_tr]
        y_te = y[index_te]
        y_tr = y[index_tr]
        
        # form data with polynomial degree
        x_te_poly = x_te
        x_tr_poly = x_tr
        
        if (ls==False):
            _, w = logistic_regression_penalized_gradient_descent(y_tr, x_tr_poly, gamma, lambda_)
        else:
            w = least_squares_lstsq_ver(y_tr, x_tr_poly)
        
        # calculate the loss for train and test data
        rmse_tr = calculate_loss(y_tr, x_tr_poly, w)
        rmse_te = calculate_loss(y_te, x_te_poly, w)
        #print(lambda_, rmse_te)
        losses_tr.append(rmse_tr)
        losses_te.append(rmse_te)
        
        y_pred = predict_labels(w, x_te_poly)
        accuracies.append(accuracy(y_te, y_pred))
        
        ws.append(w)
        
    if acc==False:
        loss_tr = np.median(losses_tr)
        loss_te = np.median(losses_te)
        return loss_tr, loss_te, np.mean(ws, axis=0)
    else:
        return np.mean(accuracies), np.mean(ws, axis=0)

In [89]:
def select_best_hypers(y, x, k_fold, seed=1):
    #y_sub, x_sub = get_subsample(y, x, 100000, seed)
    lambdas = np.logspace(-8, 0, 8)
    gammas = np.logspace(-6, 0, 5)
    #lambdas=[1e-06]
    k_indices = build_k_indices(y, k_fold, seed)
    loss_min = np.inf
    gamma_star = 0
    lambda_star = 0
    w_star = 0
    for gamma in gammas:
        for lambda_ in lambdas:
            #print(degree)
            loss_tr, loss_te, w = cross_validation(y, x, k_indices, k_fold, lambda_, gamma)
            #print(loss_te)
            if loss_te < loss_min:
                loss_min = loss_te
                print("New loss: {}, gamma: {}, lambda: {}".format(loss_te, gamma, lambda_))
                gamma_star = gamma
                lambda_star = lambda_
                w_star = w
    return gamma_star, lambda_star, loss_min, w_star

In [90]:
k_fold = 4
gammas_star=[]
lambdas_star=[]
w_star = []
for jet in range(0, 8):
    print("jet {}: ".format(jet))
    gamma_star, lambda_star, loss, w = select_best_hypers(y_tr[jet], tX_tr[jet], k_fold, 1)
    gammas_star.append(gamma_star)
    lambdas_star.append(lambda_star)
    w_star.append(w)
    #print("jet {}: Best accuracy {}, degree: {},  lambda: {}".format(jet, acc, degree_star, lambda_star))
    print("jet {}: best loss: {}, degree: {}, lambda: {}".format(jet, loss, degree_star, lambda_star))

jet 0: 
New loss: 1179.8024473808698, gamma: 1e-06, lambda: 1e-08
New loss: 910.3755491534118, gamma: 3.1622776601683795e-05, lambda: 1e-08


C:\Users\Manuel\Anaconda3\envs\ml\lib\site-packages\numpy\core\_methods.py:75: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


jet 0: best loss: 910.3755491534118, degree: 1, lambda: 1e-08
jet 1: 
New loss: 8175.298042598759, gamma: 1e-06, lambda: 1e-08
jet 1: best loss: 8175.298042598759, degree: 1, lambda: 1e-08
jet 2: 
New loss: 468.3120611902291, gamma: 1e-06, lambda: 1e-08
New loss: 404.87080616488157, gamma: 3.1622776601683795e-05, lambda: 1e-08
jet 2: best loss: 404.87080616488157, degree: 1, lambda: 1e-08
jet 3: 
New loss: 8267.720289705223, gamma: 1e-06, lambda: 1e-08
jet 3: best loss: 8267.720289705223, degree: 1, lambda: 1e-08
jet 4: 
New loss: 253.01828315110697, gamma: 1e-06, lambda: 1e-08
New loss: 240.01456169090176, gamma: 3.1622776601683795e-05, lambda: 1e-08
jet 4: best loss: 240.01456169090176, degree: 1, lambda: 1e-08
jet 5: 
New loss: 6438.087913350116, gamma: 1e-06, lambda: 1e-08
jet 5: best loss: 6438.087913350116, degree: 1, lambda: 1e-08
jet 6: 
New loss: 79.17604593049067, gamma: 1e-06, lambda: 1e-08
New loss: 75.435635555198, gamma: 3.1622776601683795e-05, lambda: 1e-08
New loss: 70.

In [103]:
#cell for logistic
y_pred = build_predictions(tX_te, indexes_te, list_w, logistic=True) 
acc = accuracy(y_te_tot, y_pred)
print(acc)

0.7347


In [106]:
def logistic_regression_penalized_stochastic_gradient_descent(y, x):
    # init parameters
    max_iter = 100000
    threshold = 1e-8
    gamma = 0.01
    lambda_ = 0
    losses = []

    # build tx
    #tx = np.c_[np.ones((y.shape[0],)), x]
    tx = x
    w = np.ones((tx.shape[1],))

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        for yn, xn in batch_iter(y, tx, 1):
            loss, w = learning_by_penalized_gradient(yn, xn, w, gamma, lambda_)
        # log info
        if iter % (max_iter/10) == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    #print("loss={l}".format(l=calculate_loss(y, tx, w)))
    return loss, w

In [ ]:
list_loss = []
list_w = []
for jet in range(0,8):
    loss, w = logistic_regression_penalized_stochastic_gradient_descent(y_tr[jet], tX_tr[jet])
    list_loss.append(loss)
    list_w.append(w)

Current iteration=0, loss=0.0297742969658984
Current iteration=10000, loss=0.03137302994548615
Current iteration=20000, loss=2.471711010496425
Current iteration=30000, loss=1.920859391350473
Current iteration=40000, loss=0.033479439964708854
Current iteration=50000, loss=0.04393166282696928
Current iteration=60000, loss=0.05107161947091559
Current iteration=70000, loss=0.35392440286158827
Current iteration=80000, loss=0.03576038158839374
Current iteration=90000, loss=0.008286371997806815
Current iteration=0, loss=0.45161063043105
Current iteration=10000, loss=1.1463531416850152
Current iteration=20000, loss=1.3015596608068494
Current iteration=30000, loss=0.18368833586275585
Current iteration=40000, loss=1.5456626692426985
Current iteration=50000, loss=1.3390517724981246
Current iteration=60000, loss=0.614621496617525
Current iteration=70000, loss=0.2752303036980852
Current iteration=80000, loss=0.10706932516817681
Current iteration=90000, loss=0.23056923299577464
Current iteration=0, 

In [94]:
#cell for logistic
y_pred = build_predictions(tX_te, indexes_te, list_w, logistic=True) 
acc = accuracy(y_te_tot, y_pred)
print(acc)

0.66852


In [38]:
def learning_by_penalized_newton_method(y, tx, w, gamma, lambda_):
    """
    Do one step on Newton's method.
    return the loss and updated w.
    """
    loss, gradient, hessian = penalized_logistic_regression_newton_method(y, tx, w, lambda_)
    w = w - gamma * np.linalg.inv(hessian).dot(gradient)
    return loss, w

In [46]:
def logistic_regression_penalized_stochastic_newton_method(y, x):
    # init parameters
    max_iter = 1000
    threshold = 1e-8
    gamma = 0.01
    lambda_ = 0.00001
    losses = []

    # build tx
    #tx = np.c_[np.ones((y.shape[0],)), x]
    tx = x
    w = np.ones((tx.shape[1],))

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        for yn, xn in batch_iter(y, tx, 10):
            loss, w = learning_by_penalized_newton_method(yn, xn, w, gamma, lambda_)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    #print("loss={l}".format(l=calculate_loss(y, tx, w)))
    return loss, w

In [47]:
list_loss = []
list_w = []
for jet in range(0,8):
    loss, w = logistic_regression_penalized_stochastic_newton_method(y_tr[jet], tX_tr[jet])
    list_loss.append(loss)
    list_w.append(w)

Current iteration=0, loss=61.163505199107995
Current iteration=100, loss=4.495607085124813
Current iteration=200, loss=6.008608529547024
Current iteration=300, loss=4.340153518666623
Current iteration=400, loss=4.4714533021184195
Current iteration=500, loss=3.2827953213383276
Current iteration=600, loss=4.082524019965291
Current iteration=700, loss=3.4382800842043415
Current iteration=800, loss=0.2766099375315662
Current iteration=900, loss=2.697715091604993
Current iteration=0, loss=26.663539751757227
Current iteration=100, loss=7.9030892799918
Current iteration=200, loss=5.204082968295155
Current iteration=300, loss=9.722157288835515
Current iteration=400, loss=7.466749452980252
Current iteration=500, loss=14.622667541512756
Current iteration=600, loss=4.063807413810574
Current iteration=700, loss=8.542093994134419
Current iteration=800, loss=3.1285467370552444
Current iteration=900, loss=7.330349870474667
Current iteration=0, loss=27.957074090843985
Current iteration=100, loss=21.33

In [48]:
#cell for logistic
y_pred = build_predictions(tX_te, indexes_te, list_w, logistic=True) 
acc = accuracy(y_te_tot, y_pred)
print(acc)

0.66538


## Generate predictions and save ouput in csv format for submission:

#### Use Accuracy and F1 Score on our classification:

In [183]:
#cell for logistic
y_pred = build_predictions(tX_te, indexes_te, list_w,logistic=True) 


In [184]:
y_pred

array([0., 0., 1., ..., 0., 0., 0.])

In [185]:
acc = accuracy(y_te_tot, y_pred)
print(acc)

0.7516


#### Build submission

In [186]:
DATA_TEST_PATH = 'test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
tX_test_new, indexes_test_new, means_test, stds_test = preprocessing([], tX_test, test=True) #same function as train,

In [190]:
#cell for logistic
y_pred_test = build_predictions(tX_test_new, indexes_test_new, list_w, logistic=True)
OUTPUT_PATH = 'submission.csv'
create_csv_submission(ids_test, y_pred_test, OUTPUT_PATH, logistic=True) #logistic flags convert [0, 1] to [-1, 1]

In [12]:
#we will need something like this

from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(tX_test_s)
plt.scatter(principalComponents[:,0], principalComponents[:,1])

NameError: name 'tX_test_s' is not defined